# Classification: Art Recognition with TensorFlow and Inception v3/ResNet50

In this example we'll classify an image with the fine-tuned Inception v3 / ResNet50 ArtNet model.


### Set-up

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import tensorflow as tf

# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# display plots in this notebook
%matplotlib inline

# set display defaults
#plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

### Helper: Load Graph

In [ ]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

### Helper: Read tensor From Image File

In [ ]:
def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
  else:
    image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0)
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result

### Helper: Load labels

In [ ]:
def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

### Set Parameters & Load Graph

In [ ]:
model_file = "./models/output_graph.pb"
label_file = "./models/output_labels.txt"
input_height = 224
input_width = 224
input_mean = 0
input_std = 255
input_layer = "Placeholder"
output_layer = "final_result"

graph = load_graph(model_file)


### Test Data

In [ ]:
fileNames = ['./dataPaperTest/1500_TheTransfiguration_Raphael.jpeg',
             './dataPaperTest/1653_Rembrandt_Aristotle_with_a_Bust_of_Homer.jpg',
             './dataPaperTest/1715_Jean-Antoine Watteau_FetesVentiennes.jpeg',
            './dataPaperTest/1812_Jacques-Louis David_Napoleon.jpeg',
            './dataPaperTest/1833_Joseph_Mallord_William_Turner_-_A_Disaster_at_Sea.jpg',
            './dataPaperTest/1869_Edgar Degas_Interior.jpeg',
            './dataPaperTest/1857_Hiroshige_One Hundred Famous Views of Edo _Yoroi Ferry in Koami-cho.jpeg',
            './dataPaperTest/1888_VanGogh_the-rocks.jpg'
            ]

### Run Tests

In [ ]:
for file_name in fileNames:
      t = read_tensor_from_image_file(
          file_name,
          input_height=input_height,
          input_width=input_width,
          input_mean=input_mean,
          input_std=input_std)

      input_name = "import/" + input_layer
      output_name = "import/" + output_layer
      input_operation = graph.get_operation_by_name(input_name)
      output_operation = graph.get_operation_by_name(output_name)

      with tf.Session(graph=graph) as sess:
        results = sess.run(output_operation.outputs[0], {
            input_operation.outputs[0]: t
        })
      results = np.squeeze(results)

      top_k = results.argsort()[-3:][::-1]  ## -9 was changed from -5 
      labels = load_labels(label_file)
      print("###############################################")
      print('file Name: ' + file_name) 
    
      for i in top_k:
        percent = results[i] * 100
        percent = '%05.2f' % (percent,)    
        print(labels[i].title() + ": " + str(percent) + "%")

## Examine intermediate output

### Final Probability Output

In [ ]:
fileNames = ['./dataPaperTest/1812_Jacques-Louis David_Napoleon.jpeg']
image=mpimg.imread(fileNames[0])
plt.imshow(image)

In [ ]:
for file_name in fileNames:
      t = read_tensor_from_image_file(
          file_name,
          input_height=input_height,
          input_width=input_width,
          input_mean=input_mean,
          input_std=input_std)

      input_name = "import/" + input_layer
      output_name = "import/" + output_layer
      input_operation = graph.get_operation_by_name(input_name)
      output_operation = graph.get_operation_by_name(output_name)

      with tf.Session(graph=graph) as sess:
        results = sess.run(output_operation.outputs[0], {
            input_operation.outputs[0]: t
        })
      results = np.squeeze(results)

      top_k = results.argsort()[-9:][::-1]  ## -9 was changed from -5 
      labels = load_labels(label_file)
      print(labels)
        
      plt.figure(figsize=(15, 3))
      plt.plot(results)  

In [ ]:
results